In [1]:
# General import of torch.
import torch
# Import for graph blocks of torch.
import torch.nn as nn
# Import the models library, to get the model to be used.
import torchvision.models as models
# Import optim library, to get the optimizer to be used.
import torch.optim as optim
# Import torchvision, to manage the input data.
import torchvision
# To apply transformations to the data (when loaded).
import torchvision.transforms as transforms

# General imports.
import os
import time
import wandb
import numpy as np
import matplotlib.pyplot as plt

### Enviroment configuration.

In [2]:
# Input size of the model.
inputSize = 'inputSize'
# Output size of the model.
outputSize = 'outputSize'
# Batch size.
batchSize = 'batchSize'
# Epochs amount.
epochs = 'epochs'
# Learning rate.
learningRate = 'learningRate'
# Examples to be showed when requested.
testView = 'testView'
# Class names.
classes = 'classes'
# Number of classes to classify.
classesLen = 'classesLen'

config = {
    inputSize    : 224,
    outputSize   : 4,
    batchSize    : 100,
    epochs       : 3,
    learningRate : 0.001,
    testView     : 8,
    classes      : ['COVID', 'Lung Opacity', 'Normal', 'Viral Pneumonia'],
    classesLen   : 4
}

In [3]:
# Device to be used, prefer cuda, if available.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Get the model.
# Get a predefined model from pytorch, and get the pretrained parameters.
net = models.resnet34(pretrained=True)
# Get the input size of the las layer of the model.
llInputSize = net.fc.in_features
# Modify the last layer of the model, to classify the amount of required classes.
net.fc = nn.Linear(llInputSize, config[outputSize])
# Load the model to the selected device.
net.to(device)

# Get criterion and optimizer
# Optimizer and the loss funtion used to train the model.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(net.parameters(), lr=config[learningRate])

# Paths of data.
#dataPath = 'data/pp/bf'
dataPath = 'data/pp/raw'

# Run name (for wanbd)
runName = 'firstCompleteTest'

### Miscellaneous functions.

In [4]:
# This functions process an metrics result dictionary for wandb. Is necessary to indicte
#   the metrics origin, training or testing.
def processMetricsWandb(metricsResults, training=False):
    # Get the prefix to log on wandb, the keys must be different.
    resultsType = 'training' if training else 'testing'

    # All the wandb keys are based in the original metrics results keys.
    lossKey = '{} ({})'.format(_loss, resultsType)
    accuracyKey = '{} ({})'.format(_accuracy, resultsType)
    accuracyClassKeys = ['{} accuracy ({})'.format(_class, resultsType) for _class in config[classes]]

    # Make the dictionary for wandb and store the values.
    wandbDict = {
        lossKey     : metricsResults[_loss].item(),
        accuracyKey : metricsResults[_accuracy].item()
    }
    for i in range(config[classesLen]):
        wandbDict[accuracyClassKeys[i]] = metricsResults[_accuracyClass][i].item()

    # Return, to log later.
    return wandbDict

# Get the metrics dictionaries for wandb and log them.
def logMetricsWandb(trainMetricsResults, testMetricsResults):
    # Get both dictionaries for wandb.
    wandbTrainDict = processMetricsWandb(trainMetricsResults, training=True)
    wandbTestDict  = processMetricsWandb(testMetricsResults, training=False)

    # Merge the dictionaries.
    wandbDict = {**wandbTrainDict, **wandbTestDict}

    # Log on wandb
    wandb.log(wandbDict)

# Pretty print the metrics dictionaries.
def printMetricsDict(metricsResults):
    # Build accuracy by class.
    accuracyClassStr = ''
    for i, _class in enumerate(config[classes]):
        accuracyClassStr += '{}: {:2.2f}%'.format(_class, metricsResults[_accuracyClass][i] * 100)
        accuracyClassStr += ', '
    accuracyClassStr = accuracyClassStr[:-2]

    print('Loss: {:.4f}, Accuracy: {:2.2f}% ({})'.format(metricsResults[_loss], metricsResults[_accuracy] * 100, accuracyClassStr))

# About metrics.
# Metric dictionary keys 
_loss            = 'Loss'
_accuracy        = 'Accuracy'
_accuracyClass   = 'Accuracy class'
_confusionMatrix = 'Confusion matrix'

# Get a clean dictionary for the metrics.
def getMetricsDict():
    return {
        _loss            : torch.tensor(0.),
        _accuracy        : torch.tensor(0.),
        _accuracyClass   : torch.zeros(config[classesLen]),
        _confusionMatrix : torch.zeros((config[classesLen], config[classesLen]), dtype=torch.int)
    }

# Function used to update the dictionary of resulting metrics.
def updateRunningMetrics(outputs, groundtruth, loss, batchAmount, metricsResults):
    # Accumulate the loss.
    metricsResults[_loss] += loss.cpu() / batchAmount
    # Accumulate the confusion matrix.
    confusionMatrix = getConfusionMatrix(outputs, groundtruth)
    metricsResults[_confusionMatrix] += confusionMatrix

# Function used to process the dictionary of resulting metrics (make final calculations).
def processRunningMetrics(metricsResults):
    # Get the total of samples processed by class.
    classTotal = torch.sum(metricsResults[_confusionMatrix], 1)
    # Get the total of samples correctly classified by class.
    classCorrect = torch.diagonal(metricsResults[_confusionMatrix])

    # Get the total accuracy, correct total samples / total samples.
    metricsResults[_accuracy] = torch.sum(classCorrect) / torch.sum(classTotal)
    # Get the total accuracy, by class.
    metricsResults[_accuracyClass] = classCorrect / classTotal

### Loader function.
Should return the training loader and test loader, a iterable object by batches.

In [5]:
# Function used to get the data loaders.
# A folder with two folders inside called train and test is expected as a rootPath.
def getLoaders(rootPath):
    # Transformation definitions.
    transformTrain = transforms.Compose([
            transforms.RandomResizedCrop(config[inputSize]),  # This one does a resize (it cuts randomly, it doesn't keep the whole image).
            transforms.RandomHorizontalFlip(),                # Flip the image horizontally randomly.
            transforms.ToTensor(),                            # Make the image a tensor.
            transforms.Normalize(
                [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Expected normalization for pretrained pytorch models.
        ])
    transformTest = transforms.Compose([
            transforms.Resize(config[inputSize]),             # Resize the image, keeping all pixels.
            transforms.CenterCrop(config[inputSize]),         # Cut the image in the center.
            transforms.ToTensor(),                            # Make the image a tensor.
            transforms.Normalize(
                [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Expected normalization for pretrained pytorch models.
        ])

    trainPath = os.path.join(rootPath, 'train')
    testPath  = os.path.join(rootPath, 'test')

    # Get the training and test data, apply the transformations.
    trainset = torchvision.datasets.ImageFolder(root=trainPath, transform=transformTrain)
    testset  = torchvision.datasets.ImageFolder(root=testPath,  transform=transformTest)

    # Get the loaders, to iterate the data through batches.
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=config[batchSize], shuffle=True, num_workers=2)
    testloader  = torch.utils.data.DataLoader(testset,  batch_size=config[batchSize], shuffle=True, num_workers=2)

    return trainloader, testloader

### Calculate the confusion matrix.

In [6]:
# Function to get the confusion matrix values.
def getConfusionMatrix(outputs, groundtruth):
    # Init the confusion matrix.
    confusionMatrix = torch.zeros((config[classesLen], config[classesLen]), dtype=torch.int)

    # Obtain the predictions (the greater number, because we use a one hot vector).
    _, predicted = torch.max(outputs, 1)

    # Iterate the predictions.
    for i in range(predicted.shape[0]):
        # Add 1 based on the prediction done for a specific label.
        confusionMatrix[groundtruth[i]][predicted[i]] += 1

    return confusionMatrix

### Training method.
This method takes care of a single training pass. Another function call this one multiple times.

In [7]:
def trainEpoch(dataloader, model, criterion, optimizer):
    # Metrics for training.
    metricsResults = getMetricsDict()

    # Enable the grad, for training.
    with torch.set_grad_enabled(True):

        # Indicate that the model is going to be trained.
        model.train()

        # Loader len, for metrics calculation.
        loaderLen = len(dataloader)

        # Iterate the batches for training.
        for batch in dataloader:
            # Train the model.
            # Get the inputs and labels, and move them to the selected device.
            inputs, labels = batch[0].to(device), batch[1].to(device)
            # Zero the gradient parameters.
            optimizer.zero_grad()
            # Get the predictions.
            outputs = model(inputs)
            # Calculate the error.
            loss = criterion(outputs, labels)
            # Calculates the derivatives of the parameters that have a gradient.
            loss.backward()
            # Update the parameters based on the computer gradient.
            optimizer.step()
            # Metrics for the training set.
            updateRunningMetrics(outputs, labels, loss, loaderLen, metricsResults)

    return metricsResults

### Evaluation method.
This method evaluates the model for a specified dataset.

In [8]:
def evaluate(dataloader, model, criterion):
    # Metrics for testing.
    metricsResults = getMetricsDict()

    # Enable the grad, for training.
    with torch.set_grad_enabled(False):

        # Indicate that the model is going to be evaluated.
        model.eval()

        # Loader len, for metrics calculation.
        loaderLen = len(dataloader)

        # Iterate the batches for testing.
        for batch in dataloader:
            # Test the model.
            # Get the inputs and labels, and move them to the selected device.
            inputs, labels = batch[0].to(device), batch[1].to(device)
            # Get the predictions.
            outputs = model(inputs)
            # Calculate the error.
            loss = criterion(outputs, labels)
            # Metrics for the testing set.
            updateRunningMetrics(outputs, labels, loss, loaderLen, metricsResults)

    return metricsResults

### Trainining and evaluate method.
For the specific purpose of this project, in each epoch we evaluate metrics for each data set (training and testing) in each epoch, this method simplifies the process. 

In [9]:
def trainAndEvaluate(trainloader, testloader, model, criterion, optimizer):

    startTimeTotal = time.time()

    for epoch in range(1, config[epochs] + 1):
        
        # Train the model.
        trainMetricsResults = trainEpoch(trainloader, model, criterion, optimizer)
        processRunningMetrics(trainMetricsResults)

        # Evaluate the model.
        testMetricsResults = evaluate(testloader, model, criterion)
        processRunningMetrics(testMetricsResults)

        # Log on wandb
        logMetricsWandb(trainMetricsResults, testMetricsResults)

        # Print the results.
        print('**', '[', 'Epoch ', epoch, ']', '*' * 48, sep='')
        print('\tTraining results:', end=' ')
        printMetricsDict(trainMetricsResults)
        print('\t Testing results:', end=' ')
        printMetricsDict(testMetricsResults)
        
    # Print time
    print('Epochs terminados')
    print("--- %s seconds ---" % (time.time() - startTimeTotal))

In [10]:
# Get the loaders.
trainloader, testloader = getLoaders(dataPath)

# Init wandb
run = wandb.init(project='CNN', entity='tecai', config=config, name=runName)
#wandb.watch(net)

# Train and evaluate
trainAndEvaluate(trainloader, testloader, net, criterion, optimizer)

# Finish wandb
run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pablobrenes (use `wandb login --relogin` to force relogin)
/home/pablo/miniconda3/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


**[Epoch 1]************************************************
	Training results: Loss: 0.4536, Accuracy: 83.40% (COVID: 74.18%, Lung Opacity: 79.69%, Normal: 89.55%, Viral Pneumonia: 78.20%)
	 Testing results: Loss: 0.2864, Accuracy: 88.97% (COVID: 78.95%, Lung Opacity: 93.78%, Normal: 92.21%, Viral Pneumonia: 68.99%)
**[Epoch 2]************************************************
	Training results: Loss: 0.2821, Accuracy: 89.75% (COVID: 86.78%, Lung Opacity: 85.16%, Normal: 93.72%, Viral Pneumonia: 88.22%)
	 Testing results: Loss: 0.3750, Accuracy: 86.27% (COVID: 63.41%, Lung Opacity: 78.94%, Normal: 99.36%, Viral Pneumonia: 81.40%)
Run pip install nbformat to save notebook history
**[Epoch 3]************************************************
	Training results: Loss: 0.2389, Accuracy: 91.41% (COVID: 89.89%, Lung Opacity: 87.25%, Normal: 94.53%, Viral Pneumonia: 90.43%)
	 Testing results: Loss: 0.3990, Accuracy: 85.38% (COVID: 75.65%, Lung Opacity: 71.89%, Normal: 99.85%, Viral Pneumonia: 61.2

Loss (training),0.2389
Accuracy (training),0.91407
COVID accuracy (training),0.89893
Lung Opacity accuracy (training),0.87245
Normal accuracy (training),0.94528
Viral Pneumonia accuracy (training),0.90432
Loss (testing),0.39895
Accuracy (testing),0.85377
COVID accuracy (testing),0.75653
Lung Opacity accuracy (testing),0.71891
Normal accuracy (testing),0.99853


Loss (training),█▂▁
Accuracy (training),▁▇█
COVID accuracy (training),▁▇█
Lung Opacity accuracy (training),▁▆█
Normal accuracy (training),▁▇█
Viral Pneumonia accuracy (training),▁▇█
Loss (testing),▁▇█
Accuracy (testing),█▃▁
COVID accuracy (testing),█▁▇
Lung Opacity accuracy (testing),█▃▁
Normal accuracy (testing),▁██


In [11]:
# Guardar el modelo.
#PATH = './cifar_net.pth'
#torch.save(net.state_dict(), PATH)

# Cargar el modelo.
# Esta linea de abajo debe coincidir con el modelo que voy a usar.
#   instancio el mismo modelo que guardé
# net = Net()
# net.load_state_dict(torch.load(PATH))